## Custom Stock Screener

Import Libraries, ignore warnings and set working directories

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import os
from datetime import datetime


#dont show warnings
import warnings

warnings.filterwarnings("ignore")

data_directory = os.getcwd() + "/data/"

### Read the csv file with the stocks

You can add to this file any stock you want just by adding a new line in the csv file (it can be only the symbol)

In [ ]:
stocks = pd.read_csv(data_directory + "sp500.csv")
stocks.head()

### Get information from Yahoo Finance

Loop through the stocks and get information from Yahoo Finance
During the loop the information of each stock will be stored in a list
If there is an error we just print it. Most probable case is that the symbol cannot be found in yahoo finance at least the way is in our list

In [ ]:
yf_info = []
err_counter = 0

for index, row in stocks.iterrows():
    try:
        ticker = yf.Ticker(row["Symbol"])
        yf_info.append(ticker.info)
    except Exception as e:
        print(f'Something went wrong with symbol {row["Symbol"]} with error {e}')
        err_counter += 1
    
screener = pd.DataFrame(yf_info)
print(f'There were {err_counter} errors')
print(f'Got the information for {len(screener)} stocks')

## Work with the dataframe

First we check if there are any missing values and we get the ones with the most

In [ ]:
missing_values = screener.isnull().sum()
sorted_missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
print(sorted_missing_values.head(10))





### We drop all the columns that mostly contain missing values

In [ ]:
cols_to_drop = ['zip','phone','address1', 'address2','state', 'industrySymbol', 'fax']
screener = screener.drop(cols_to_drop, errors='ignore', axis=1)
screener.head()

### We also must check which stocks have the most missing values

First we will get an ordered list of the number of missing values per row

In [ ]:
screener['missing_values'] = screener.isna().sum(axis=1)
# Sort the DataFrame based on the 'missing_values' column
screener.sort_values(by='missing_values', ascending=False)

We see that there are stocks that have a lot of missing values - we will set a threashold to 80

In [ ]:
screener = screener[screener['missing_values'] < 80]
screener = screener.drop('missing_values', axis=1)

### Now we examine the columns that exists in the dataframe

In [ ]:
screener.columns.tolist()

## Following the review we can drop collumns that we believe unessesary for a stock screener

In [ ]:
cols_to_drop = ['zip','phone','address1', 'state']
screener = screener.drop(cols_to_drop, errors='ignore', axis=1)

## We will convert the epoch variables to datetime format

In [ ]:
epoch_columns = ['governanceEpochDate', 'compensationAsOfEpochDate', 'exDividendDate', 'lastSplitDate', 'lastDividendDate', 'firstTradeDateEpochUtc']
screener[epoch_columns] = screener[epoch_columns].apply(pd.to_datetime, unit='s')

# List of old column names and corresponding new column names
epoch_columns_to_rename = ['old_col1', 'old_col2', 'old_col3']
epoch_columns_new_name = ['new_col1', 'new_col2', 'new_col3']

# Rename columns
screener.rename(columns=dict(zip(epoch_columns_to_rename, epoch_columns_new_name)), inplace=True)

## Create a category columns based on Capitalization

In [ ]:
# Define the bins and labels for each capitalization category
bins = [0, 50_000_000, 300_000_000, 2_000_000_000, 10_000_000_000, 200_000_000_000, float('inf')]
labels = ['nano', 'micro', 'small', 'mid', 'large', 'mega']

# Create a new column with the categorized values
screener['CapCategory'] = pd.cut(screener['marketCap'], bins=bins, labels=labels, right=False)

screener['CapCategory'].value_counts()

## Calculate how close is the current price to the high and low of 52 weeks

In [ ]:
screener['52WeeksCurrentPosition'] = (screener['currentPrice'] - screener['fiftyTwoWeekLow']) / (screener['fiftyTwoWeekHigh'] -screener['fiftyTwoWeekLow'])

## Calculate some metrics on the C Level executives of the company

In [ ]:
def get_company_officers(row):
    # we check in case is not a list to return NaN and not throw an error
    if not isinstance(row['companyOfficers'], list):
        return None, None
    return len(row['companyOfficers']), round(np.mean([obj['age'] for obj in row['companyOfficers'] if 'age' in obj and obj['age'] is not None]), 2)
    
screener[['CountOfCompanyOfficers','AvgAgeOfCompanyOfficers']] = screener.apply(lambda row: pd.Series(get_company_officers(row)), axis=1)
screener[['symbol', 'shortName', 'CountOfCompanyOfficers','AvgAgeOfCompanyOfficers']].head()

## Because yfinance does not provide a meaningfull order for the dataframe we are going to move in the beginning the collumns that makes sense to be first  


In [ ]:
cols_to_move_beginning = ['symbol', 'shortName', 'sector', 'industry','CapCategory']
screener = screener[cols_to_move_beginning + [col for col in screener.columns if col not in cols_to_move_beginning]]
screener.head()

## Last we save the dataframe in a csv format that we can open in an excel and examine further

The csv file is saved with a prefix of the current date and time - this way you can keep track of all the information you have acquired in one place in the past


In [ ]:
formatted_datetime = datetime.now().strftime('%Y%m%d%H%M')
screener.to_csv(f'{data_directory}{formatted_datetime} Custom Screener.csv', index=False)